Script to post-process high-resolution WRF model output.

Major tasks include computing the following for selected variables:
  1. domain-averages to produce time series
  2. vertical integrals
  3. pressure-level vertical interpolation

James Ruppert  
25 Nov 2024

In [1]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, ALL_TIMES#, vinterp
import xarray as xr
from read_wrf_piccolo import *
from post_proc_functions import *
import os

In [2]:
do_2d_special = True
do_3d_vars = False
do_3d_special = False

In [3]:
########################################################
# Directories and test selection
########################################################

datdir = "/glade/derecho/scratch/ruppert/piccolo/"
# datdir = "/glade/campaign/univ/uokl0053/"

case = "sept1-4"
test_process = "ctl"
wrf_dom = "wrf_fine"
nmem = 5 # number of ensemble members

########################################################
# New functions and single-use calls go here
########################################################

# Get variable lists
vars2d = var_list_2d()
# vars3d = var_list_3d()

# Ens-member string tags (e.g., memb_01, memb_02, etc.)
memb0=1 # Starting member to read
memb_nums_str=np.arange(memb0,nmem+memb0,1).astype(str)
nustr = np.char.zfill(memb_nums_str, 2)
memb_all=np.char.add('memb_',nustr)

def memb_dir_settings(memb_dir):
    wrfdir = datdir+case+'/'+memb_dir+'/'+test_process+"/"+wrf_dom+"/"
    outdir = wrfdir+"post_proc/"
    os.makedirs(outdir, exist_ok=True)
    # Get WRF file list, dimensions
    wrffiles = get_wrf_file_list(wrfdir, "wrfout_d01*")
    # hffiles = get_wrf_file_list(wrfdir, "hfout_d01*")
    lat, lon, nx1, nx2, nz, npd = wrf_dims(wrffiles[0])
    nfiles = len(wrffiles)
    # New vertical dimension for pressure levels
    # dp = 25 # hPa
    # pres = np.arange(1000, 25, -dp)
    # nznew = len(pres)
    return outdir, wrffiles, nfiles, npd

In [4]:
def get_2d_special_vars_iwrf(file):

    ds = Dataset(file)
    print("Opened "+file)

    # qv = getvar(ds, "QVAPOR", timeidx=0)#, cache=cache)
    qv = ds.variables["QVAPOR"][0:5,:,:,:]
    return qv
    print("Got qv")
    pwrf = getvar(ds, "p", units='Pa', timeidx=0)#, cache=cache)
    print("Got pwrf")
    # hght = getvar(dset, "zstag", units='m', timeidx=ALL_TIMES)#, cache=cache)
    # tmpk = getvar(dset, "tk", timeidx=ALL_TIMES)#, cache=cache)
    # rho = density_moist(tmpk, qv, pwrf)

    # Get dz
    # dz = np.zeros(qv.shape)
    # for iz in range(nz):
    #     dz[:,iz] = hght[:,iz+1] - hght[:,iz]
    # Get dp
    dp = pwrf.differentiate('bottom_top')*-1
    print("Got dp")

    # pclass
    ipclass = wrf_pclass(ds, dp)
    print("Got pclass")
    # pw
    ipw = vert_int(qv, dp)
    print("Got pw")
    # pw_sat
    qvsat = get_rv_sat(ds, pwrf)
    print("Got qvsat1")
    qvsat = xr.DataArray(qvsat, coords=qv.coords, dims=qv.dims, attrs=qv.attrs)
    print("Got qvsat2")
    ipw_sat = vert_int(qvsat, dp)
    print("Got pw_sat")

    ds.close()

    return ipclass, ipw, ipw_sat

In [5]:
if do_2d_special:

    memb_dir = memb_all[0]

    # for memb_dir in memb_all:

    # memb_dir = memb_all[comm.rank]

    print("Processing member "+memb_dir)
    outdir, wrffiles, nfiles, npd = memb_dir_settings(memb_dir)

    # Read in variable from WRF files
    # for ifile in range(nfiles):
    for ifile in range(1):

        # Open the WRF file
        wrffile = wrffiles[ifile]

        # Process variables
        # ipclass, ipw, ipw_sat = get_2d_special_vars_iwrf(wrffile)
        qv = get_2d_special_vars_iwrf(wrffile)
        continue

        # Concatenate variables

        if ifile == 0:
            # pclass
            pclass_all = ipclass
            # pw
            pw_all = ipw
            # pw_sat
            pw_sat_all = ipw_sat
        else:
            pclass_all = xr.concat((pclass_all, ipclass), 'Time')
            pw_all = xr.concat((pw_all, ipw), 'Time')
            pw_sat_all = xr.concat((pw_sat_all, ipw_sat), 'Time')

    # Remove duplicate time steps
    # pclass_all = pclass_all.drop_duplicates(dim="Time", keep='first')
    # pw_all     = pw_all.drop_duplicates(dim="Time", keep='first')
    # pw_sat_all = pw_sat_all.drop_duplicates(dim="Time", keep='first')

    # Write out the variables
    # var_name='pclass'
    # write_ncfile(outdir, pclass_all, var_name)
    # var_name='pw'
    # write_ncfile(outdir, pw_all, var_name)
    # var_name='pw_sat'
    # write_ncfile(outdir, pw_sat_all, var_name)

print("Done writing out special 2D variables")

Processing member memb_01
Opened /glade/derecho/scratch/ruppert/piccolo/sept1-4/memb_01/ctl/wrf_fine/wrfout_d01_2024-09-01_12:00:00
Done writing out special 2D variables


In [6]:
# GB
gb = qv.nbytes/1e9
print(gb, "GB for one time step")

# for ntime = 72
print(gb*72, "GB for all 72 time steps")

3.555 GB for one time step
255.96 GB for all 72 time steps


In [14]:
ds = xr.open_dataset(wrffile)

In [16]:
gb_ds = ds.nbytes/1e9
gb_ds

1423.081389528

In [25]:
ds['QVAPOR'].nbytes / 1e9
ds['QVAPOR'].data.nbytes / 1e9

: 

In [22]:
447387324973*1e-9/(1.064)

420.4768091851504

In [ ]:
ds.close()

In [7]:
import psutil
import numpy as np

def memory_usage():

    mem = psutil.virtual_memory()

    togb = 1e-9

    nround=2

    total    = np.round(mem[0]*togb, nround)
    avail    = np.round(mem[1]*togb, nround)
    used     = np.round(mem[3]*togb, nround)
    free     = np.round(mem[4]*togb, nround)
    active   = np.round(mem[5]*togb, nround)
    inactive = np.round(mem[6]*togb, nround)

    print()
    print("Total:     ",total,"GB")
    print("Avail:     ",avail,"GB")
    print("Used:      ",used,"GB")
    print("Free:      ",free,"GB")
    print("Active:    ",active,"GB")
    print("Inactive:  ",inactive,"GB")
    print()
memory_usage()


Total:      540.27 GB
Avail:      464.66 GB
Used:       70.78 GB
Free:       420.14 GB
Active:     2.6 GB
Inactive:   29.15 GB

